In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import numpy as np 
import scipy
import pandas as pd
from matplotlib import pyplot as plt 
import seaborn as sns
import os # imports for loading data
from tqdm import tqdm

In [ ]:
# Importing the data in
df = pd.read_csv(os.path.abspath('turn_over.csv'), index_col=False) 
print(f"Successfully read in {df.size} values.")

print("First 5 values:")
df.head(5)

In [ ]:
list_Month = [i.split(" ")[0][0:2] for i in  list(df['CALENDAR_DATE'])]
list_Date = [i.split(" ")[0][6:8] for i in  list(df['CALENDAR_DATE'])]
list_Year = [i.split(" ")[0][3:5] for i in  list(df['CALENDAR_DATE'])]

df['Month'] = list_Month
df['Date'] = list_Date
df['Year'] = list_Year

In [ ]:
df['ds'] = pd.DatetimeIndex(df['Year']+'-'+df['Month']+'-'+df['Date'])

In [ ]:
df= df.rename(columns=str.lower)

In [ ]:
df = df.drop_duplicates(inplace=False,keep='first')

In [ ]:
df.reset_index(inplace=True,drop=True)

In [ ]:
df = df.groupby(['ds','cust_id','jurisdiction'])['turnover'].sum()
df = pd.DataFrame(df)
df = df.rename_axis(['ds', 'cust_id', 'jurisdiction']).reset_index()

In [ ]:
df['ds'].agg(['min', 'max'])
#334 days`

In [ ]:
df.cust_id.nunique()

In [ ]:
juris = list(set(df['jurisdiction']))
juris

In [ ]:
df1 = df[df['jurisdiction']=='DK'].reset_index(drop=True)
df2 = df[df['jurisdiction']=='BE'].reset_index(drop=True)
df3 = df[df['jurisdiction']=='MT'].reset_index(drop=True)
df4 = df[df['jurisdiction']=='IE'].reset_index(drop=True)
df5 = df[df['jurisdiction']=='DE'].reset_index(drop=True)
df6 = df[df['jurisdiction']=='NT'].reset_index(drop=True)
df7 = df[df['jurisdiction']=='SE'].reset_index(drop=True)
df8 = df[df['jurisdiction']=='EE'].reset_index(drop=True)
df9 = df[df['jurisdiction']=='RO'].reset_index(drop=True)
df10 = df[df['jurisdiction']=='IT'].reset_index(drop=True)
df11 = df[df['jurisdiction']=='UK'].reset_index(drop=True)


In [ ]:
df

In [ ]:
df['turnover'].sum()

In [ ]:
df1.head()

In [ ]:
df1.shape

In [ ]:
df1.turnover.sum()

In [ ]:
df1.cust_id.nunique()

In [ ]:
594*334

In [ ]:
def addzeros(dataframe):
    juris = dataframe['jurisdiction'][0]
    my_dates = pd.date_range('2021-07-01', '2022-05-30', freq='D')
    idx= pd.MultiIndex.from_product([my_dates, dataframe.cust_id.unique()],names=['ds', 'cust_id'])
    dataframe = dataframe.set_index(['ds', 'cust_id']).reindex(idx).fillna(0).reset_index()
    dataframe['jurisdiction'] = juris
    return (dataframe)

In [ ]:
df1 = addzeros(df1)
df2 = addzeros(df2)
df3 = addzeros(df3)
df4 = addzeros(df4)
df5 = addzeros(df5)
df6 = addzeros(df6)
df7 = addzeros(df7)
df8 = addzeros(df8)
df9 = addzeros(df9)
df10 = addzeros(df10)
df11 = addzeros(df11)

In [ ]:
dataframe = pd.concat([df1, df2,df3,df4,df5,df6,df7,df8,df9,df10,df11], ignore_index=True)

In [ ]:
dataframe

In [ ]:
dataframe.groupby('cust_id')['turnover'].sum().sort_values(ascending=False)

In [ ]:
#model implement check for two customers (to plot results in report)
#5520
customer = dataframe[dataframe["cust_id"] == 5520].reset_index()
#10081
customer1 = dataframe[dataframe["cust_id"] == 10081].reset_index()


In [ ]:
sns.set_style('darkgrid')
sns.set(rc={'figure.figsize':(8,4)})

ax = sns.lineplot(data=customer, x ='ds', y = 'turnover',
                  hue='cust_id', palette='viridis',
                  legend='full',lw =3)

plt.legend(bbox_to_anchor=(1, 1))
plt.ylabel('TURNOVER')
plt.xlabel('Year-Month')
plt.show()

In [ ]:
from statsmodels.tsa.stattools import adfuller # dickey fuller implementation

def dickey_fuller_metrics(time_series):
    print('Dickey Fuller Results:')
    dicker_fuller_test = adfuller(time_series, autolag='AIC') # perform the test
    results_table = pd.Series(dicker_fuller_test[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used']) # create a results table to print
    for key,value in dicker_fuller_test[4].items():
        results_table['Critical Value (%s)'%key] = value
    print(results_table)

dickey_fuller_metrics(customer['turnover'])

In [ ]:
customer['square_root'] = np.sqrt(customer['turnover'])
customer['square_root'] = customer['square_root'].dropna() # remove NaNs

# Plot the square root transform
fig, ax = plt.subplots();
customer['square_root'].plot(ax=ax, figsize = (8,4), color="purple" );
plt.ylabel('turnover')
plt.xlabel('square root transformation')
plt.title('Square Root Transformation of customer 5520')

In [ ]:
from statsmodels.tsa.stattools import adfuller # dickey fuller implementation

def dickey_fuller_metrics(time_series):
    print('Dickey Fuller Results:')
    dicker_fuller_test = adfuller(time_series, autolag='AIC') # perform the test
    results_table = pd.Series(dicker_fuller_test[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used']) # create a results table to print
    for key,value in dicker_fuller_test[4].items():
        results_table['Critical Value (%s)'%key] = value
    print(results_table)

dickey_fuller_metrics(customer['square_root'])

In [ ]:
customer.shape

In [ ]:
# data splitting - 80% train , 20% test
#train = 268, test = 66

forecast_train = customer['square_root'].values[:268]
forecast_test = customer['square_root'].values[268:]

# extract the times
forecast_train_times = customer['square_root'].index[:268]
forecast_test_times = customer['square_root'].index[268:]

In [ ]:
import itertools
#best value for arima 

p=d=q=range(0,5)
pdq = list(itertools.product(p,d,q))

history = forecast_train.tolist()
AR_predictions = list()

for param in pdq:
    try:
        model_arima = ARIMA(history , order = param)
        model_arima_fit = model_arima.fit()
        print(param, model_arima_fit.aic)
        
    except:
        continue
        
#2,1,3 ; 3,1,4; 2,1,4
#3,0,3

In [ ]:
##ARIMA

from statsmodels.tsa.arima.model import ARIMA 
import warnings
warnings.filterwarnings('ignore') # sometimes a convergence warning may appear, which is best to ignore to allow the notebook to look neater



history = forecast_train.tolist()
AR_predictions = list()

    # walk-forward validation
for t in tqdm(range(len(forecast_test))):
        AR_model = ARIMA(history, order=(3,0,3))
        AR_model_fit = AR_model.fit()
        output = AR_model_fit.forecast()
        yhat = output[0]
        AR_predictions.append(yhat)
        obs = forecast_test[t]
        history.append(obs)# timing how long it takes for the model to run using tqdm
        

In [ ]:
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# Plot the results
plt.figure(figsize=(10,7), dpi=100)
plt.plot(forecast_train_times, forecast_train, label='Training Data')
plt.plot(forecast_test_times, forecast_test.flatten(), label='Actual Value')
plt.plot(forecast_train_times, AR_model_fit.fittedvalues[:268], label='Fitted')
plt.plot(forecast_test_times, AR_predictions, label='Forecasted')
#plt.title('AR Model RSS: %.4f' %sum((AR_model_fit.fittedvalues[:268] - forecast_train.ravel())**2))
plt.title('ARIMA model (3,0,3) for customer id - 1 ')
plt.legend(loc='upper left', fontsize=8)

In [ ]:
mse= (sum((AR_model_fit.fittedvalues[:268] - forecast_train.ravel())**2))/66

In [ ]:
#mse

In [ ]:
##customer 1

In [ ]:
sns.set_style('darkgrid')
sns.set(rc={'figure.figsize':(8,4)})

ax = sns.lineplot(data=customer1, x ='ds', y = 'turnover',
                  hue='cust_id', palette='viridis',
                  legend='full',lw =3)

plt.legend(bbox_to_anchor=(1, 1))
plt.ylabel('TURNOVER')
plt.xlabel('Year-Month')
plt.show()

In [ ]:
customer1['square_root'] = np.sqrt(customer1['turnover'])
customer1['square_root'] = customer1['square_root'].dropna() # remove NaNs

# Plot the square root transform
fig, ax = plt.subplots();
customer1['square_root'].plot(ax=ax, figsize = (8,4), color="purple" );
plt.ylabel('turnover')
plt.xlabel('square root transformation')
plt.title('Square Root Transformation of customer 10081')

In [ ]:
from statsmodels.tsa.stattools import adfuller # dickey fuller implementation

def dickey_fuller_metrics(time_series):
    print('Dickey Fuller Results:')
    dicker_fuller_test = adfuller(time_series, autolag='AIC') # perform the test
    results_table = pd.Series(dicker_fuller_test[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used']) # create a results table to print
    for key,value in dicker_fuller_test[4].items():
        results_table['Critical Value (%s)'%key] = value
    print(results_table)

dickey_fuller_metrics(customer1['square_root'])

In [ ]:
customer.shape

In [ ]:
# data splitting - 80% train , 20% test
#train = 268, test = 66

forecast_train = customer1['square_root'].values[:268]
forecast_test = customer1['square_root'].values[268:]

# extract the times
forecast_train_times = customer1['square_root'].index[:268]
forecast_test_times = customer1['square_root'].index[268:]

In [ ]:
##ARIMA

from statsmodels.tsa.arima.model import ARIMA 
import warnings
warnings.filterwarnings('ignore') # sometimes a convergence warning may appear, which is best to ignore to allow the notebook to look neater



history = forecast_train.tolist()
AR_predictions = list()

    # walk-forward validation
for t in tqdm(range(len(forecast_test))):
        AR_model = ARIMA(history, order=(3,0,3))
        AR_model_fit = AR_model.fit()
        output = AR_model_fit.forecast()
        yhat = output[0]
        AR_predictions.append(yhat)
        obs = forecast_test[t]
        history.append(obs)# timing how long it takes for the model to run using tqdm
        

In [ ]:
## 7 0 0 - 587
##200 587


from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# Plot the results
plt.figure(figsize=(10,7), dpi=100)
plt.plot(forecast_train_times, forecast_train, label='Training Data')
plt.plot(forecast_test_times, forecast_test.flatten(), label='Actual Value')
plt.plot(forecast_train_times, AR_model_fit.fittedvalues[:268], label='Fitted')
plt.plot(forecast_test_times, AR_predictions, label='Forecasted')
#plt.title('AR Model RSS: %.4f' %sum((AR_model_fit.fittedvalues[:268] - forecast_train.ravel())**2))
plt.title('ARIMA (3,0,3) for customer id - 2')
plt.legend(loc='upper left', fontsize=8)

In [ ]:
mse= (sum((AR_model_fit.fittedvalues[:268] - forecast_train.ravel())**2))/66

In [ ]:
## for all customers in df4- done for each jurisdiction
##could be extended for every juridiction

cust = df4.cust_id.unique()

mselist = []

for c in cust:
    
    customer = df4[df4["cust_id"] == c].reset_index()
    customer['square_root'] = np.sqrt(customer['turnover'])
    customer['square_root'] = customer['square_root'].dropna()
    
    forecast_train = customer['square_root'].values[:268]
    forecast_test = customer['square_root'].values[268:]

    # extract the times
    forecast_train_times = customer['square_root'].index[:268]
    forecast_test_times = customer['square_root'].index[268:]
    
    history = forecast_train.tolist()
    AR_predictions = list()

    # walk-forward validation
    for t in tqdm(range(len(forecast_test))):
            AR_model = ARIMA(history, order=(2,1,4))
            AR_model_fit = AR_model.fit()
            output = AR_model_fit.forecast()
            yhat = output[0]
            AR_predictions.append(yhat)
            obs = forecast_test[t]
            history.append(obs)
            
    m = (sum((AR_model_fit.fittedvalues[:268] - forecast_train.ravel())**2))/66
    mselist.append(m)
    

    

In [ ]:
# import statistics
# statistics.mean(mselist)